In [1]:
# Add locations of modules
push!(LOAD_PATH, "../src/")

4-element Array{String,1}:
 "@"      
 "@v#.#"  
 "@stdlib"
 "../src/"

In [2]:
# Required packages
using LinearAlgebra
using Convex, SCS, Mosek, MosekTools

using GraphicalModelLearning
using LightGraphs

# Function definitions
using Printf, SparseArrays
include("beta_scaling.jl")
include("active_learning.jl")

using DelimitedFiles

In [3]:
using Revise

In [4]:
# parameters
FLAG_create_struct_gm = true
FLAG_regular_random_gm = false
FLAG_lattice_gm = true
FLAG_weak_impurity = true
sampling_regime = M_regime()
learning_algo = NLP()

┌ Warning: `with_optimizer` is deprecated. Adapt the following example to update your code:
│ `with_optimizer(Ipopt.Optimizer, max_cpu_time=60.0)` becomes `optimizer_with_attributes(Ipopt.Optimizer, "max_cpu_time" => 60.0)`.
│   caller = ip:0x0
└ @ Core :-1


NLP(MathOptInterface.OptimizerWithAttributes(Ipopt.Optimizer, Pair{MathOptInterface.AbstractOptimizerAttribute,Any}[RawParameter("print_level")=>0]))

In [5]:
if FLAG_lattice_gm
    # text file to save adjacency matrix in
    file_adj_matrix_gm = "adj_matrix_ferro_lattice_gm_E_MEE.txt"
    # name of picture to save graphical model in
    file_plot_gm = "ferro_lattice_gm_E_MEE.eps"
    # File to save final results
    file_M_opt_gm = "M_opt_Ferro_Lattice_E_MEE.txt"
elseif FLAG_regular_random_gm
    # text file to save adjacency matrix in
    file_adj_matrix_gm = "adj_matrix_spin_glass_gm.txt"
    # name of picture to save graphical model in
    file_plot_gm = "spin_glass_gm3.eps"
    # File to save final results
    file_M_opt_gm = "M_opt_M_Regime_Regular_Random_B.txt"
end

"M_opt_Ferro_Lattice_E_MEE.txt"

In [6]:
# Simple test
N = 16
d = 4
α = 0.4
β_array = [1.0+0.5*i for i=1:5]

# Create and plot the initial graphical graphical model
β = copy(β_array[1])
adj_matrix, struct_adj_matrix = ferro_lattice(N,α,β,FLAG_weak_impurity)
m = n = Int(sqrt(N))
open(file_adj_matrix_gm, "w") do io
    writedlm(io, adj_matrix)
end;


In [7]:
## Start the complexity studies
τ = α/2
L = 45
M_g = 4000
M_factor = 0.05

M_opt = Array{Int64,1}(undef,length(β_array))

5-element Array{Int64,1}:
 4973202032
 4973202160
 4973202224
 4973202288
 4542625952

In [9]:
for i = 1:length(β_array)
    # define β
    β = copy(β_array[i])
    println(β)

    if i==1
        M_g = 6000
    end
    println(M_g)
    
    # Create the adjacency matrix
    adj_matrix, struct_adj_matrix = ferro_lattice(N,α,β,FLAG_weak_impurity)

    # Get the optimum number of samples
    M_opt[i] = get_M_opt_glauber_dynamics_AL(adj_matrix, τ, L, M_g, M_factor)

    # Update the guess of M_opt
    M_g = Int(floor(0.8*M_opt[i]))
end

1.5
6000


┌ Info: assuming spin alphabet
└ @ GraphicalModelLearning /Users/gogol/Dropbox (Personal)/Research/LANL/Code/GML_Glauber_Dynamics.jl/src/models.jl:108
┌ Info: assuming spin alphabet
└ @ GraphicalModelLearning /Users/gogol/Dropbox (Personal)/Research/LANL/Code/GML_Glauber_Dynamics.jl/src/models.jl:108
┌ Info: using Glauber dynamics v1 to generate M-regime samples
└ @ GraphicalModelLearning /Users/gogol/Dropbox (Personal)/Research/LANL/Code/GML_Glauber_Dynamics.jl/src/sampling.jl:909
┌ Info: using JuMP for RISE to learn Glauber dynamics
└ @ GraphicalModelLearning /Users/gogol/Dropbox (Personal)/Research/LANL/Code/GML_Glauber_Dynamics.jl/src/GraphicalModelLearning.jl:368
┌ Info: using Glauber dynamics v1 to generate M-regime samples
└ @ GraphicalModelLearning /Users/gogol/Dropbox (Personal)/Research/LANL/Code/GML_Glauber_Dynamics.jl/src/sampling.jl:909
┌ Info: using JuMP for RISE to learn Glauber dynamics
└ @ GraphicalModelLearning /Users/gogol/Dropbox (Personal)/Research/LANL/Code/GML_Gl

FLAG=0, M=6300, n_trials=0, P=0/45


┌ Info: assuming spin alphabet
└ @ GraphicalModelLearning /Users/gogol/Dropbox (Personal)/Research/LANL/Code/GML_Glauber_Dynamics.jl/src/models.jl:108
┌ Info: using Glauber dynamics v1 to generate M-regime samples
└ @ GraphicalModelLearning /Users/gogol/Dropbox (Personal)/Research/LANL/Code/GML_Glauber_Dynamics.jl/src/sampling.jl:909
┌ Info: using JuMP for RISE to learn Glauber dynamics
└ @ GraphicalModelLearning /Users/gogol/Dropbox (Personal)/Research/LANL/Code/GML_Glauber_Dynamics.jl/src/GraphicalModelLearning.jl:368


InterruptException: InterruptException:

In [54]:
#mtot = 300000
#mtot = 315000
mtot = 15750

15750

In [57]:
#m0 = Int(floor(mtot/3))
m0 = 5000
mbatch = min(2000,Int(floor((mtot-m0)/15)))

716

In [58]:
m_remaining = mtot-m0
M = copy(m0)

while m_remaining > 0
    # modify the distribution slightly by adding a small amount of uniform distribution
    lambda = 1 - 1/((M)^(1/6))
    
    mtemp = min(m_remaining,mbatch)
    
    m_remaining = m_remaining - mtemp
    M += mtemp

    # add to the existing set of samples
    @printf("lambda=%f,m_remaining=%d,M=%d\n",lambda,m_remaining,M)
end

lambda=0.758173,m_remaining=10034,M=5716
lambda=0.763507,m_remaining=9318,M=6432
lambda=0.768113,m_remaining=8602,M=7148
lambda=0.772157,m_remaining=7886,M=7864
lambda=0.775753,m_remaining=7170,M=8580
lambda=0.778987,m_remaining=6454,M=9296
lambda=0.781919,m_remaining=5738,M=10012
lambda=0.784600,m_remaining=5022,M=10728
lambda=0.787065,m_remaining=4306,M=11444
lambda=0.789346,m_remaining=3590,M=12160
lambda=0.791466,m_remaining=2874,M=12876
lambda=0.793445,m_remaining=2158,M=13592
lambda=0.795299,m_remaining=1442,M=14308
lambda=0.797043,m_remaining=726,M=15024
lambda=0.798688,m_remaining=10,M=15740
lambda=0.800244,m_remaining=0,M=15750


In [48]:
M_factor = 0.05

0.05

In [49]:
M = Int(floor(((1 + M_factor)*M)/50))*50

330750

In [59]:
function solve_exp_system(β_array::Array,M_array::Array,d_max::Integer)
    # Set up the least squares problem
    b = log.(M_array)
    A = ones(length(β_array),2)
    for i=1:length(β_array)
        A[i,2] = β_array[i]*d_max
    end
    x = A\b
    return x
end

solve_exp_system (generic function with 1 method)

In [ ]:
# M-Regime 
β_array = vcat([0.4+0.1*i for i=0:27],[3.5,4.0,4.5,5.0])
M_array = readdlm("FigE_M_Results.txt")

β_array_US = [1.5,1.6,1.7,2.0,2.2,2.5,2.7]
M_array_US = [15750,22050,24000,45000,57500,120000,178650]

yticks_array = 10.^(4:6)
xticks_array = 0.0:0.5:3.0

d_max = 4

x = solve_exp_system(β_array[7:end], M_array[7:end], d_max)
x_US = solve_exp_system(β_array_US, M_array_US, d_max)

c1 = "#911eb4" #Purple
c2 = "#3cb44b" #Green

figure(figsize=(7.5,4.5))
ax=subplot(111)
ax[:spines]["top"][:set_color]("none") # Remove the top axis boundary
ax[:spines]["right"][:set_color]("none") # Remove the right axis boundary    


# Curve Fit
end_plot=27
semilogy(β_array[7:end_plot],exp(x[1])*exp.(x[2]*β_array[7:end_plot]*d_max),linestyle="--",c=c1,linewidth=2,label=latexstring("\$\\exp(",string(round(x[2],2)),"\\beta d)\$"),zorder=1)
semilogy(β_array_US,exp(x_US[1])*exp.(x_US[2]*β_array_US*d_max),linestyle="--",c=c2,linewidth=2,label=latexstring("\$\\exp(",string(round(x_US[2],2)),"\\beta d)\$"),zorder=1)

# Data
scatter(β_array[1:end_plot],M_array[1:end_plot],s=30,color=c1,marker="s",edgecolor="k",linewidth=0.5, zorder=2, label="GD-RISE")
scatter(β_array_US,M_array_US,s=30,color=c2,marker="o",edgecolor="k",linewidth=0.5, zorder=2, label="GD-RISE with AL")

xlabel(L"\beta",fontsize=16)
ylabel(L"m^{\star}",fontsize=16)
legend(fontsize=14, frameon=false,loc=2)
xticks(xticks_array,fontsize=14)
yticks(yticks_array, fontsize=14)
#grid("on",linestyle="dotted")

savefig("FigE_M_GDRISE_AL_Results.png", format="png", dpi=300,bbox_inches="tight")